В этом домашнем задании необходимо подготовить данные и

написать SQL запрос который будет работать оптимальным образом

на любых данных. Оценка будет рассчитана по производительности и по качеству.

Запрос будет запущен 10 раз с разными параметрами (по три раза с каждым из параметров, чтобы исключить выбросы)

за каждое совпадение ответа с baseline начисляется максимум 1 балл, в зависимости от производительности вашего решения. Чем быстрее от baseline тем ближе к 1 баллу за 1 итерацию запуска. Максимум 10 баллов.

ВНИМАНИЕ: Список тестовых жанров (genres) в финальном тесте будет отличаться от того, что в ноутбуке. Ваш запрос должен работать на любых значениях жанров, аналогично тому, как работает baseline запрос.

За списывание - 0 баллов, срок сдачи **27.05.2024 00:00:00 MSK**, -1 бал до **28.05.2024 00:00:00 MSK**, далее 0 баллов

Ноутбук c решением **экспортируется** File/Dowload/ipynb и отправляется email со вложением на адрес ilya+hse@aniskovets.com

В ноутбуке, пожалуйста, не изменяйте ничего, кроме блока инициализации и вашего ответа.

Ноутбук должен запускаться без ошибок из меню Runtime/Run All

Ссылки на ноутбуки не принимаются, необходимо физически экспортировать ноутбук и отправить на email ilya+hse@aniskovets.com

**Полезные ссылки**:

https://spark.apache.org/docs/latest/sql-ref.html

как создавать таблицы с партицированием и бакетированием:

https://spark.apache.org/docs/latest/sql-ref-syntax-ddl-create-table-hiveformat.html

Документация по схемам БД:

https://developer.imdb.com/non-commercial-datasets/



In [1]:
!pip install pyspark==3.5.1

In [2]:
!curl -O https://mars.ru77.ru/data/title.basics.tsv.gz
!curl -O https://mars.ru77.ru/data/title.ratings.tsv.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  132M  100  132M    0     0  8379k      0  0:00:16  0:00:16 --:--:-- 11.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5526k  100 5526k    0     0  1079k      0  0:00:05  0:00:05 --:--:-- 1304k


In [3]:
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkConf, SparkContext

spark = SparkSession.builder.master("local[8]").config("spark.driver.memory", "8g").appName("hse").enableHiveSupport().getOrCreate()
sql = spark.sql

In [4]:
title_basics_csv = spark.read.csv("title.basics.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_basics_csv.createOrReplaceTempView("title_basics_csv")

title_ratings_csv = spark.read.csv("title.ratings.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_ratings_csv.createOrReplaceTempView("title_ratings_csv")


In [5]:
_ = spark.sql("""
CREATE TABLE IF NOT EXISTS baseline_title_basics USING PARQUET AS SELECT * FROM title_basics_csv
""").collect()
_ = spark.sql("""
CREATE TABLE IF NOT EXISTS baseline_title_ratings USING PARQUET AS SELECT * FROM title_ratings_csv
""").collect()


In [6]:
import time
import random

query_1_baseline = """
   -- это baseline, не изменяйте его
    SELECT tb.*
    FROM baseline_title_basics tb
    JOIN baseline_title_ratings tr ON tb.tconst = tr.tconst
    WHERE tr.averageRating > {averageRating} AND tr.numVotes >= {minVotes} AND tb.startYear >= {startYear} and array_contains(split(genres, '[,]'), {genre})
"""

In [7]:
genres = spark.sql("""
select distinct explode(split(genres, '[,]'))
FROM baseline_title_basics
""").collect()

In [8]:
def validate_data(baseline_df, answer_df):
  baseline_df_count = baseline_df.count()
  print("baseline_df_count:", baseline_df_count)

  answer_df_count = answer_df.count()
  print("answer_df_count:", answer_df_count)

  assert(baseline_df_count == answer_df_count)

  baseline_df_distinct = baseline_df.distinct()
  answer_df_distinct = answer_df.distinct()

  baseline_df_distinct_count = baseline_df_distinct.count()
  print("baseline_df_distinct_count:", baseline_df_distinct_count)

  answer_df_distinct_count = answer_df_distinct.count()
  print("answer_df_distinct_count:", answer_df_distinct_count)

  assert(baseline_df_distinct_count == answer_df_distinct_count)

  merged_df_distinct_count = baseline_df.unionByName(answer_df).distinct().count()
  print("merged_df_distinct_count: ", merged_df_distinct_count)

  assert(merged_df_distinct_count == baseline_df_distinct_count)


На стадии инициализации вы можете создавать любые таблицы, эта стадия не учитывается при расчете производительности решения. В данном случае мы создаем таблицы в формате паркет, как пример такой оптимизации. Запрос будет запускаться с различными случайными параметрами, результат должен совпадать с baseline.

ВНИМАНИЕ: Список тестовых жанров (genres) в финальном тесте будет отличаться от того, что в ноутбуке. Ваш запрос должен работать на любых значениях жанров, аналогично тому, как работает baseline запрос.

**В ячейке ниже необходимо написать ваш инициализационный код, ниже только пример такого кода**

**В ячейке ниже необходимо написать ваш SQL запрос**




In [9]:
_ = spark.sql("""
CREATE TABLE IF NOT EXISTS title_basics USING PARQUET AS SELECT *
FROM title_basics_csv ORDER BY tconst
""").collect()

_ = spark.sql("""
CREATE TABLE IF NOT EXISTS title_ratings USING PARQUET AS SELECT *
FROM title_ratings_csv ORDER BY tconst
""").collect()


query_1_answer = """
WITH filtered_ratings (
  SELECT tr.tconst
  FROM title_ratings tr
  WHERE tr.averageRating > {averageRating}
    AND tr.numVotes >= {minVotes}
),
filtered_titles (
  SELECT
    tb.tconst,
    tb.primaryTitle,
    tb.titleType,
    tb.originalTitle,
    tb.startYear,
    tb.isAdult,
    tb.endYear,
    tb.runtimeMinutes,
    tb.genres
  FROM title_basics tb
  WHERE tb.startYear >= {startYear}
    AND FIND_IN_SET({genre}, tb.genres) > 0
)
SELECT
    tb.tconst,
    tb.primaryTitle,
    tb.titleType,
    tb.originalTitle,
    tb.startYear,
    tb.isAdult,
    tb.endYear,
    tb.runtimeMinutes,
    tb.genres
FROM filtered_titles tb
JOIN filtered_ratings tr
  ON tb.tconst = tr.tconst;
"""

Второй вариант решения

In [ ]:
spark.sql("""
CREATE TABLE title_basics_ratings
USING PARQUET
PARTITIONED BY (genres_exploded, startYear)
CLUSTERED BY (averageRating, numVotes)
SORTED BY (averageRating ASC, numVotes ASC)
INTO 8 BUCKETS
AS
SELECT tb.tconst,
       tb.titleType,
       tb.primaryTitle,
       tb.originalTitle,
       tb.endYear,
       CAST(tb.isAdult AS TINYINT) AS isAdult,
       CAST(tb.startYear AS SMALLINT) AS startYear,
       CAST(tb.runtimeMinutes AS INT) AS runtimeMinutes,
       tb.genres AS genres,
       explode(split(tb.genres, '[,]')) AS genres_exploded,
       CAST(tr.averageRating AS DOUBLE) AS averageRating,
       CAST(tr.numVotes AS INT) AS numVotes
FROM title_basics tb
JOIN title_ratings tr ON tb.tconst = tr.tconst;
""")
spark.sql("""SELECT * FROM title_basics_ratings;""").show()

**Это автотест, пожалуйста, не изменяйте его**


In [10]:
iterations = 10

attempts = []

for i in range(iterations):
  attempt = {}
  print("iteration:", i)
  attempt['averageRating'] = random.random() * 10
  attempt['minVotes'] = random.randint(100, 5000)
  attempt['startYear'] = random.randint(1800, 2028)
  attempt['genre'] = genres[random.randint(0, len(genres) - 1)][0]

  baseline_df = spark.sql(query_1_baseline, **attempt)
  answer_df = spark.sql(query_1_answer, **attempt)

  start = time.time()
  _ = baseline_df.limit(10).collect()
  end = time.time()
  attempt['baseline_time_1'] = end - start

  start = time.time()
  _ = answer_df.limit(10).collect()
  end = time.time()
  attempt['answer_time_1'] = end - start

  start = time.time()
  _ = baseline_df.limit(10).collect()
  end = time.time()
  attempt['baseline_time_2'] = end - start

  start = time.time()
  _ = answer_df.limit(10).collect()
  end = time.time()
  attempt['answer_time_2'] = end - start

  start = time.time()
  _ = baseline_df.limit(10).collect()
  end = time.time()
  attempt['baseline_time_3'] = end - start

  start = time.time()
  _ = answer_df.limit(10).collect()
  end = time.time()
  attempt['answer_time_3'] = end - start

  print(attempt)
  attempts.append(attempt)

  validate_data(baseline_df, answer_df)

iteration: 0
{'averageRating': 0.8740046303809013, 'minVotes': 3998, 'startYear': 1826, 'genre': 'Romance', 'baseline_time_1': 2.7245559692382812, 'answer_time_1': 1.7748775482177734, 'baseline_time_2': 2.334035634994507, 'answer_time_2': 1.5263960361480713, 'baseline_time_3': 1.4890921115875244, 'answer_time_3': 1.117569923400879}
baseline_df_count: 3145
answer_df_count: 3145
baseline_df_distinct_count: 3145
answer_df_distinct_count: 3145
merged_df_distinct_count:  3145
iteration: 1
{'averageRating': 8.744991758161534, 'minVotes': 530, 'startYear': 1937, 'genre': 'Short', 'baseline_time_1': 4.545339107513428, 'answer_time_1': 5.71745228767395, 'baseline_time_2': 4.052622318267822, 'answer_time_2': 3.818162202835083, 'baseline_time_3': 4.821440696716309, 'answer_time_3': 3.6557137966156006}
baseline_df_count: 36
answer_df_count: 36
baseline_df_distinct_count: 36
answer_df_distinct_count: 36
merged_df_distinct_count:  36
iteration: 2
{'averageRating': 9.25409928513767, 'minVotes': 3422,

In [11]:
import csv
import sys

def print_array_of_dicts_as_csv(array_of_dicts):
    if len(array_of_dicts) == 0:
        return

    fieldnames = array_of_dicts[0].keys()
    writer = csv.DictWriter(sys.stdout, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(array_of_dicts)

print_array_of_dicts_as_csv(attempts)

averageRating,minVotes,startYear,genre,baseline_time_1,answer_time_1,baseline_time_2,answer_time_2,baseline_time_3,answer_time_3
0.8740046303809013,3998,1826,Romance,2.7245559692382812,1.7748775482177734,2.334035634994507,1.5263960361480713,1.4890921115875244,1.117569923400879
8.744991758161534,530,1937,Short,4.545339107513428,5.71745228767395,4.052622318267822,3.818162202835083,4.821440696716309,3.6557137966156006
9.25409928513767,3422,1915,Film-Noir,9.889217615127563,6.412600994110107,10.14216947555542,5.309187412261963,10.319328546524048,4.8073506355285645
9.852567253127141,265,1920,Talk-Show,10.443947076797485,6.606718301773071,9.927922487258911,5.465392589569092,10.234130859375,4.71635103225708
8.951017609559646,787,1811,Game-Show,11.718353748321533,5.692245960235596,10.50520396232605,5.206521987915039,10.171860456466675,4.846303701400757
0.056800280066003905,3453,1921,News,4.138132095336914,3.686460256576538,4.674758434295654,3.704981565475464,3.394226551055908,3.4259822368621826